In [30]:
# Exam 2020/06/27 - Exercise #2 - SQL-based version #1

In [3]:
inputPathCars = "exam_ex2_data/Cars.txt"
inputPathFailures = "exam_ex2_data/CarsFailures.txt"
outputPathPart1 = "outPart1"
outputPathPart2 = "outPart2"

In [4]:
# *****************************************
# Exercise 2 - Part 1
# *****************************************

In [7]:
# Read the content of CarsFailures.txt
failuresDF = spark.read.load(inputPathFailures,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "date")\
.withColumnRenamed("_c1", "time")\
.withColumnRenamed("_c2", "carId")\
.withColumnRenamed("_c3", "failureType")\
.cache()

In [15]:
# Read the content of Cars.txt
carsDF = spark.read.load(inputPathCars,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "carId")\
.withColumnRenamed("_c1", "model")\
.withColumnRenamed("_c2", "company")\
.withColumnRenamed("_c3", "city")

In [16]:
# Associate the two dataFrames to two temporary tables
carsDF.createOrReplaceTempView("cars")
failuresDF.createOrReplaceTempView("failures")

In [17]:
# Define a UDF that returns 1 if the year is 2018. 0 otherwise.
def year2018Func(date):
    if date.startswith("2018"):
        return 1
    else:
        return 0
    

spark.udf.register("year2018", year2018Func)

<function __main__.year2018Func(date)>

In [18]:
# Define a UDF that returns 1 if the year is 2017. 0 otherwise.
def year2017Func(date):
    if date.startswith("2017"):
        return 1
    else:
        return 0
    

spark.udf.register("year2017", year2017Func)

<function __main__.year2017Func(date)>

In [24]:
# Select only the failures related to years 2017 and 2018 and failureType=Engine
# Count 
# - the number of engine failures in year 2017
# - the number of engine failures in year 2018 
# Select only the car with #failures 2018 > #failures 2017

selectedCarsModelsDF = spark.sql("""SELECT cars.carId, model
FROM cars, failures
WHERE cars.carId=failures.carId
AND failures.failureType='Engine'
GROUP BY cars.carId, model
HAVING SUM( year2018(date)) > SUM( year2017(date)) """)

In [ ]:
# Store the result

In [26]:
selectedCarsModelsDF.write.csv(outputPathPart1,header=False)

In [ ]:
# *****************************************
# Exercise 2 - Part 2
# *****************************************

In [49]:
# This function is already provided 
from datetime import datetime, timedelta
def previousDate(mydate):
    currentDate=datetime.strptime(mydate,"%Y/%m/%d")
    prevDate=currentDate-timedelta(days=1)
    return prevDate.strftime("%Y/%m/%d")

In [50]:
# Select only carId and Date from each failure
# and remove duplicates
carsDatesDF = failuresDF.select("carId", "Date").distinct()

In [58]:
# Transform carsDatesDF into an RDD.
# Map the returned Row object to pairs (only for simplicity/for reusing the same code used for the 
# solution based only on  RDDs for the next part)
carsDatesDistinctRDD = carsDatesDF.rdd\
.map(lambda row: (row["carId"], row["Date"]))

In [53]:
# Emit two pairs from each input pair
# - ((carId,  date), +1)   - First element of the window that starts in this date
# - ((carId,date-1), +1)   - Second element of the window that starts in the previous date
def flatMapFunc(pair):
    returnedPairs = []
    
    carId = pair[0]
    currDate = pair[1]
    prevDate= previousDate(currDate)
    
    returnedPairs.append( ((carId, currDate), +1) )
    returnedPairs.append( ((carId, prevDate), +1) )
    
    return returnedPairs


windowsElementsRDD = carsDatesDistinctRDD.flatMap(flatMapFunc)

In [55]:
# Count the number of elements per windows
windowsNumElementsRDD = windowsElementsRDD.reduceByKey(lambda v1, v2: v1+v2)

In [56]:
# Select the windows with 2 elements
selectedWindowsRDD = windowsNumElementsRDD.filter(lambda pair: pair[1]==2)

In [57]:
# Store the key part of the selected pairs (carId, first date of the sequence)
selectedWindowsRDD.keys().saveAsTextFile(outputPathPart2)